In [ ]:
!pip install -q pdf2image pillow numpy nltk bert-score openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.2 MB/s eta 0:00:00


In [ ]:
# Code Cell 1: Imports and Setup
# -------------------------------
import os
import re
import base64
import json
import time
import functools
import traceback

from pdf2image import convert_from_path, pdf2image # Import specific exception if needed
from PIL import Image
import numpy as np

# NLP and Grading specific imports
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
# Ensure necessary NLTK data is downloaded (run once)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt_tab')
# Note: 'punkt_tab' was in the original grading code imports, but might not be standard.
# If errors occur, uncomment the next lines:
# try:
#     nltk.data.find('tokenizers/punkt_tab')
# except nltk.downloader.DownloadError:
#     nltk.download('punkt_tab')

# Sentence Transformers (if used directly, though BERTScore is primary here)
# from sentence_transformers import SentenceTransformer

# BERTScore for evaluation
# Make sure bert-score is installed: pip install bert-score
import bert_score # Main import
# from bert_score import score as bert_score_calc # Can use alias if preferred

# External API Client (OpenRouter in this case)
from openai import OpenAI # Use OpenAI client library for compatible APIs

# Environment Variables for API Key

print("Imports successful.")
print(f"BERTScore version: {bert_score.__version__}")

# --- Environment Setup ---
# Load API key from .env file if present
# IMPORTANT: Set your API Key here or in a .env file (OPENROUTER_API_KEY=sk-...)
OPENROUTER_API_KEY = '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' # Replace default if needed
# Initialize API Client (using OpenAI library structure for OpenRouter)
# New
try:
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=OPENROUTER_API_KEY
    )
    # Optional: Test connection if API provides a simple ping/model list endpoint
    # client.models.list() # Example check if available
    print("OpenRouter client initialized.")
except Exception as e:
    print(f"Error initializing OpenRouter client: {e}")
    client = None # Set client to None if initialization fails

# --- Model Configuration ---
# New - Define the VL model to use (as specified in original code)
VL_MODEL_NAME = "meta-llama/llama-4-maverick:free" # "moonshot/moonshot-v1-128k" was another option, stick to original unless specified
print(f"Using VL Model: {VL_MODEL_NAME}")

# Ensure poppler is installed and in PATH for pdf2image, especially on Windows
# You might need to specify the path explicitly:
# POPPLER_PATH = r"C:\path\to\poppler-xx.xx.x\bin" # Example Windows path
# if os.name == 'nt' and 'POPPLER_PATH' in locals():
#     print(f"Setting poppler path to: {POPPLER_PATH}")
#     os.environ["PATH"] += os.pathsep + POPPLER_PATH

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Imports successful.
BERTScore version: 0.3.12
OpenRouter client initialized.
Using VL Model: meta-llama/llama-4-maverick:free


In [ ]:
# Code Cell 2: Configuration - Input Files and Grading Parameters
# ---------------------------------------------------------------

# --- Input File Paths ---
# Path to the question paper (must be an image file like JPEG, PNG)
QUESTION_PAPER_PATH = "/content/question_paper.jpeg" # Replace with your actual path

# Path to the student's answer sheet (can be PDF or an image file/list of image files)
ANSWER_SHEET_PATH = "/content/Joel-D041.pdf" # Replace with your actual path (PDF or image)
# Example for multiple images:
# ANSWER_SHEET_PATH = ["page1.jpg", "page2.jpg"]

# Path to the answer key (optional, can be PDF, image, list of images, or 'dict')
ANSWER_KEY_PATH = "/content/Spam_AnswerKey.pdf" # Replace with actual path or set ANSWER_KEY_SOURCE to 'dict'
# Example for multiple images:
# ANSWER_KEY_PATH = ["key_page1.jpg", "key_page2.jpg"]

# Define how the Answer Key is provided:
# 'file': Process ANSWER_KEY_PATH using VL model OCR.
# 'dict': Use the predefined dictionary ANSWER_KEY_DICT below.
ANSWER_KEY_SOURCE = 'file' # Change to 'file' if using a file

# --- Predefined Answer Key (if ANSWER_KEY_SOURCE = 'dict') ---
# Keys MUST be integers corresponding to question numbers.
ANSWER_KEY_DICT = {
    1: """
Asymptotic notations are mathematical tools used to describe the running time or space complexity of algorithms:

1. Big-O Notation (O):
   - Represents the upper bound/worst-case complexity
   - f(n) = O(g(n)) means f(n) grows no faster than g(n)
   - Example: Bubble Sort has O(n²) in worst case

2. Omega Notation (Ω):
   - Represents the lower bound/best-case complexity
   - f(n) = Ω(g(n)) means f(n) grows at least as fast as g(n)
   - Example: Bubble Sort has Ω(n) when array is already sorted

3. Theta Notation (Θ):
   - Represents both upper and lower bounds (tight bound)
   - f(n) = Θ(g(n)) means f(n) grows at the same rate as g(n)
   - Example: Matrix multiplication has Θ(n³)

Key Points:
- Big-O is most commonly used for worst-case analysis
- Ω is useful for understanding best-case scenarios
- Θ gives precise average-case analysis for balanced algorithms
""",
    2: """
Operations on Circular Linked List:

1. Insertion at Start:
   newNode = createNode(data)
   if head == NULL:
       head = newNode
       newNode.next = head
   else:
       temp = head
       while temp.next != head:
           temp = temp.next
       temp.next = newNode
       newNode.next = head
       head = newNode

2. Deletion of Last Node:
   if head == NULL: return
   if head.next == head:
       free(head)
       head = NULL
   else:
       current = head
       while current.next.next != head:
           current = current.next
       free(current.next)
       current.next = head

Example:
Initial: 1 → 2 → 3 → (back to 1)
After inserting 0 at start: 0 → 1 → 2 → 3 → (back to 0)
After deleting last node: 0 → 1 → 2 → (back to 0)
""",
    3: """
Infix to Postfix Conversion Table for: A + B * (C - D) ^ E / (F - G + H * I)

| Symbol | Stack (bottom to top) | Output | Action Taken |
|--------|-----------------------|--------|--------------|
| A      | []                    | A      | Output operand |
| +      | [+]                   | A      | Push operator |
| B      | [+]                   | AB     | Output operand |
| *      | [+, *]                | AB     | Push operator (higher precedence than +) |
| (      | [+, *, (]             | AB     | Push parenthesis |
| C      | [+, *, (]             | ABC    | Output operand |
| -      | [+, *, (, -]          | ABC    | Push operator |
| D      | [+, *, (, -]          | ABCD   | Output operand |
| )      | [+, *]                | ABCD-  | Pop until '(' |
| ^      | [+, *, ^]             | ABCD-  | Push operator (highest precedence) |
| E      | [+, *, ^]             | ABCD-E | Output operand |
| /      | [+, *, /]             | ABCD-E^| Pop ^ (higher precedence), then push / |
| (      | [+, *, /, (]          | ABCD-E^| Push parenthesis |
| F      | [+, *, /, (]          | ABCD-E^F | Output operand |
| -      | [+, *, /, (, -]       | ABCD-E^F | Push operator |
| G      | [+, *, /, (, -]       | ABCD-E^FG | Output operand |
| +      | [+, *, /, (, +]       | ABCD-E^FG- | Pop -, push + (same precedence) |
| H      | [+, *, /, (, +]       | ABCD-E^FGH | Output operand |
| *      | [+, *, /, (, +, *]    | ABCD-E^FGH | Push operator (higher precedence than +) |
| I      | [+, *, /, (, +, *]    | ABCD-E^FGHI | Output operand |
| )      | [+, *, /]             | ABCD-E^FGHI*+- | Pop until '(' |
| End    | []                    | ABCD-E^FGHI*+-/ | Pop all remaining operators |

Final Postfix Expression: ABCD-E^*FGHI*+-/+
"""
}

# --- Grading Rubrics/Marks ---
# Keys MUST be integers corresponding to question numbers.
MARKS_PER_QUESTION = {
    1: 3,
    2: 7,
    3: 5,
    # Add max marks for all questions evaluated
}

# --- PDF to Image Conversion Output Folder ---
# New - Made configurable
IMAGE_OUTPUT_FOLDER = "temp_images_output"

# --- Validation ---
# Check if input files exist (basic check)
if not os.path.exists(QUESTION_PAPER_PATH):
    print(f"ERROR: Question paper image not found at: {QUESTION_PAPER_PATH}")
# Add checks for ANSWER_SHEET_PATH and ANSWER_KEY_PATH if ANSWER_KEY_SOURCE=='file'
# ... (add checks as needed)

print("Configuration loaded.")

Configuration loaded.


In [ ]:
# Code Cell 3: Helper Functions (PDF Conversion, Synonyms, Similarity)
# --------------------------------------------------------------------

# From OCRandSegment.ipynb
def pdf_to_images(pdf_path, output_folder="images"):
    """Convert PDF pages to image files."""
    if not os.path.exists(pdf_path):
        print(f"Error: PDF file not found at {pdf_path}")
        return []
    try:
        os.makedirs(output_folder, exist_ok=True)
        print(f"Converting PDF '{os.path.basename(pdf_path)}' to images in '{output_folder}'...")
        # Note: Explicitly passing poppler_path might be needed on some systems
        # images = convert_from_path(pdf_path, poppler_path=POPPLER_PATH)
        images = convert_from_path(pdf_path)
        image_paths = []
        for i, image in enumerate(images):
            path = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(pdf_path))[0]}_page_{i+1}.jpg")
            image.save(path, "JPEG")
            image_paths.append(path)
        print(f"Converted PDF to {len(image_paths)} image(s).")
        return image_paths
    # except pdf2image.exceptions.PDFInfoNotInstalledError:
    #     print("ERROR: pdfinfo command not found. Ensure Poppler is installed and in your PATH.")
    #     print("See: https://pdf2image.readthedocs.io/en/latest/installation.html")
    #     return []
    except Exception as e:
        print(f"Error converting PDF '{pdf_path}' to images: {e}")
        print(traceback.format_exc())
        return []

# From Marks_Accuracy_model1.ipynb
@functools.lru_cache(maxsize=128)
def get_synonyms(word):
    """Get synonyms for a given word with caching for performance."""
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace("_", " "))
    return synonyms

# From Marks_Accuracy_model1.ipynb
def similar(a, b):
    """Check if two strings are similar using multiple metrics."""
    a, b = a.lower(), b.lower()

    # Direct match or substring check
    if a == b or a in b or b in a:
        return True

    # Character similarity (Jaccard similarity) - prevent division by zero
    a_chars, b_chars = set(a), set(b)
    intersection = len(a_chars.intersection(b_chars))
    union = len(a_chars.union(b_chars))
    if union == 0:
      return True if intersection == 0 else False # Both empty strings are similar

    return intersection / union > 0.7 # Threshold from original code

In [ ]:
# Code Cell 4: Vision Language Model (VL) OCR and Structuring Functions
# -----------------------------------------------------------------------
# Based on OCRandSegment.ipynb, using the configured VL Model via OpenRouter client

# From OCRandSegment.ipynb
def extract_text_from_question_paper(image_path):
    """
    Use VL model to extract text from question paper image with specific formatting.
    Returns structured question paper text or None on error.
    """
    if not client:
        print("ERROR: OpenRouter client not initialized. Cannot call API.")
        return None
    if not os.path.exists(image_path):
        print(f"ERROR: Question paper image not found at {image_path}")
        return None

    print(f"Extracting text from Question Paper: {os.path.basename(image_path)} using {VL_MODEL_NAME}...")
    try:
        with open(image_path, "rb") as img_file:
            base64_image = base64.b64encode(img_file.read()).decode("utf-8")

        # Using the specific prompt from the original code
        completion = client.chat.completions.create(
            model=VL_MODEL_NAME,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": (
                                "You are an OCR expert specialized in academic question papers. "
                                "Extract all the visible text from this question paper image exactly as it appears. "
                                "Follow these specific guidelines:\n"
                                "1. Start with 'QUESTIONS:'\n"
                                "2. For each question, start with 'QUESTION X:' where X is the question number\n"
                                "3. Include marks in square brackets [X] where present\n"
                                "4. Preserve 'OR' sections exactly as they appear\n"
                                "5. Maintain all mathematical expressions, symbols, and formatting\n"
                                "6. Do not add any explanations or interpretations\n"
                                "7. Do not include any thought process or internal reasoning\n"
                                "8. Preserve exact spacing and line breaks between questions\n"
                                "9. Extract any instructions or notes exactly as they appear\n"
                                "10. Maintain sub-points and numbering exactly as shown"
                            )
                        },
                        {
                            "type": "image_url",
                            "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"} # Assuming JPEG/PNG common
                        }
                    ]
                }
            ]
        )
        extracted_text = completion.choices[0].message.content

        # Clean up (same as original code) - maybe rename clean_vl_output later if needed
        cleaned_text = (
            extracted_text
            .replace("◁think▷", "")  # Remove any thinking markers if present
            .replace("\n\n\n", "\n\n") # Reduce excessive line breaks
            .strip()
        )

        # Ensure the text starts with "QUESTIONS:" (same as original)
        if not cleaned_text.startswith("QUESTIONS:"):
             cleaned_text = "QUESTIONS:\n\n" + cleaned_text # Add prefix if missing

        print("Question paper text extracted successfully.")
        return cleaned_text

    except Exception as e:
        print(f"ERROR extracting text from question paper '{os.path.basename(image_path)}': {e}")
        print(traceback.format_exc())
        return None

# From OCRandSegment.ipynb
def extract_text_from_images(image_paths, context_question_paper_text):
    """
    Use VL model to extract text from a list of answer sheet images,
    using the question paper text as context.
    Returns structured answer text or None on error.
    """
    if not client:
        print("ERROR: OpenRouter client not initialized. Cannot call API.")
        return None
    if not image_paths:
        print("ERROR: No image paths provided for text extraction.")
        return None
    if not context_question_paper_text:
        print("WARNING: Question paper text context is missing. Extraction quality may be reduced.")
        context_question_paper_text = "QUESTIONS:\n[Context not available]" # Provide placeholder

    print(f"Extracting text from {len(image_paths)} answer image(s) using {VL_MODEL_NAME}...")

    # Derive valid question numbers from the context (QP text)
    valid_questions = set()
    # Improved regex to handle various Q formats like "QUESTION 1:", "Q. 2", "Q3)" etc.
    question_matches = re.finditer(r'(?:QUESTION|Q\.?)\s*(\d+[a-zA-Z]?)\s*[:.)]?', context_question_paper_text, re.IGNORECASE)
    for match in question_matches:
         q_num_str = match.group(1)
         # Try converting to int, handle sub-questions like '3a' if needed later
         try:
            # For now, just store the string identifier found
            valid_questions.add(q_num_str)
         except ValueError:
            print(f"Warning: Could not parse question identifier '{q_num_str}' as integer.")
            valid_questions.add(q_num_str) # Keep non-integer identifiers as strings

    print(f"Derived valid question identifiers from QP: {sorted(list(valid_questions))}")
    if not valid_questions:
        print("WARNING: No valid question identifiers found in the question paper text. Segmentation might fail.")
        # Decide how to proceed: maybe attempt extraction without filtering? Or stop?
        # For now, proceed but the VL model might struggle without valid Q numbers.

    # Prepare image contents for the API call
    image_contents = []
    for idx, image_path in enumerate(image_paths):
        if not os.path.exists(image_path):
            print(f"WARNING: Image file not found: {image_path}. Skipping.")
            continue
        try:
            with open(image_path, "rb") as img_file:
                base64_image = base64.b64encode(img_file.read()).decode("utf-8")
            image_contents.append({
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"} # Assuming JPEG/PNG
            })
        except Exception as e:
            print(f"Error reading or encoding image {image_path}: {e}")

    if not image_contents:
        print("ERROR: No valid images could be prepared for API call.")
        return None

    # Create the prompt using the structure from original code
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": (
                        "You are an OCR expert specialized in academic answer papers. "
                        f"You will be analyzing {len(image_contents)} images that may contain parts of the same answers. "
                        "\nEXTRACTION RULES:"
                        "\n1. Valid question numbers/identifiers from question paper: " + ", ".join(sorted(list(valid_questions))) + # Use derived identifiers
                        f"\n\nReference Question Paper Context:\n{context_question_paper_text}\n" # Provide QP context
                        "\n2. Multi-Image Processing Rules:"
                        "\n   - Combine content from all images to create complete answers for each question number/identifier."
                        "\n   - Avoid duplicating answers for the same question number/identifier."
                        "\n   - Maintain chronological order of content as it appears across images."
                        "\n   - Ensure seamless integration of content belonging to the same answer, even if split across images."
                        "\n3. Text Extraction Guidelines:"
                        "\n   - Start each unique answer with 'QUESTION X:' (where X is one of the valid numbers/identifiers listed above)."
                        "\n   - Each valid question number/identifier should appear at most once in the final output."
                        "\n   - Maintain consistent formatting for the extracted answer text."
                        "\n   - Preserve all mathematical expressions, code blocks, tables, and symbols exactly as they appear in the answer."
                        "\n4. Critical Rules:"
                        "\n   - ONLY structure output using the valid question numbers/identifiers provided."
                        "\n   - If content cannot be associated with a valid number/identifier, ignore it."
                        "\n   - Ensure the text following 'QUESTION X:' corresponds to the answer for that question."
                        "\n   - Do not add any explanations, interpretations, or summaries."
                        "\n   - Do not include any processing notes or thinking steps (like ◁think▷)."
                        "\n   - Remove any duplicate 'QUESTION X:' tags if they accidentally occur for the same X."
                        "\nProcess all images together and provide a single coherent text output containing all the extracted answers, correctly labelled and combined."
                    )
                }
            ] + image_contents # Append the list of image objects
        }
    ]

    # Make the API call
    try:
        completion = client.chat.completions.create(
            model=VL_MODEL_NAME,
            messages=messages
            # Consider adding max_tokens if needed, though VL models often handle this well
            # max_tokens=4096 # Example
        )
        extracted_text = completion.choices[0].message.content

        # Clean up the extracted text (same cleaning as original code)
        # Rename to avoid potential future conflicts? -> clean_vl_output
        # New - Renaming this function
        def clean_vl_output(vl_text):
            """Cleans VL model specific output artifacts."""
            cleaned = vl_text
            # Remove artifacts seen in original code
            cleaned = cleaned.replace("◁think▷", "")
            cleaned = cleaned.replace("Valid Question Numbers:", "") # Remove potential meta-text
            cleaned = cleaned.replace("Question Paper Text:", "")   # Remove potential meta-text
            # Remove other common VL model preamble/postamble if observed
            cleaned = re.sub(r"^\s*Here is the extracted text.*?\n+", "", cleaned, flags=re.IGNORECASE)
            cleaned = re.sub(r"\n+Okay, I have processed the images.*$", "", cleaned, flags=re.IGNORECASE | re.DOTALL)
            cleaned = cleaned.strip()
            # Ensure consistent spacing around QUESTION tags (add newline before if missing)
            cleaned = re.sub(r'(?<!\n)\n?(QUESTION \S+:)', r'\n\n\1', cleaned)
            cleaned = re.sub(r'\n{3,}', '\n\n', cleaned) # Consolidate newlines
            return cleaned.strip()

        cleaned_text = clean_vl_output(extracted_text)

        print(f"Answer text extracted successfully from {len(image_contents)} image(s).")
        return cleaned_text

    except Exception as e:
        print(f"ERROR during VL API call for answer extraction: {e}")
        print(traceback.format_exc())
        return None


# From OCRandSegment.ipynb (preprocess_text - now specific to cleaning VL output artifacts)
# Note: This was already specific, but renaming makes it clearer.
# Kept original name here to match dependencies, but wrapped its logic in clean_vl_output above.
def preprocess_text(text):
    """
    Preprocess the raw text extracted by the VL model to remove metadata/notes.
    (This function's logic is now inside `clean_vl_output` called by `extract_text_from_images`)
    """
    # The actual cleaning happens within extract_text_from_images now using clean_vl_output.
    # This function remains mostly for compatibility if it was called elsewhere,
    # but ideally, the output from extract_text_from_images is already cleaned.
    # We can simply return the text as is, assuming it's cleaned by the caller.
    if text is None: return None
    # Or, apply the cleaning again just in case:
    return text # Assuming extract_text_from_images already returns cleaned text.


# From OCRandSegment.ipynb
def convert_to_dict(text):
    """
    Convert the structured VL text output ("QUESTION X: ...") into a dictionary.
    Keys are integers, values are the corresponding answer strings.
    Handles potential errors during parsing.
    """
    qa_dict = {}
    if not text:
        print("Warning: Cannot convert empty text to dictionary.")
        return qa_dict

    # Split text based on "QUESTION X:" pattern. Handles variations like "QUESTION 1:", "QUESTION 3a:"
    # Positive lookahead `(?=...)` ensures the delimiter is not consumed.
    # Handles potential whitespace variations.
    questions = re.split(r'(?=QUESTION\s+\S+\s*:)', text.strip(), flags=re.IGNORECASE)
    questions = [q.strip() for q in questions if q.strip()] # Filter out empty strings

    if not questions:
         print("Warning: No 'QUESTION X:' patterns found in the text to split by.")
         # Maybe return the whole text under a default key?
         # qa_dict[0] = text
         return qa_dict

    for entry in questions:
        # Match "QUESTION X:" at the beginning of the string
        match = re.match(r'QUESTION\s+(\S+)\s*:(.*)', entry, re.IGNORECASE | re.DOTALL)
        if match:
            question_id_str = match.group(1).strip()
            answer = match.group(2).strip()
            try:
                # Convert key to integer - THIS IS CRUCIAL for matching marks/key dicts
                question_id_int = int(question_id_str)
                qa_dict[question_id_int] = answer
            except ValueError:
                print(f"Warning: Could not convert question identifier '{question_id_str}' to integer. Storing as string key.")
                # Decide how to handle non-integer keys if they shouldn't occur.
                # Option 1: Store as string (might break later matching)
                qa_dict[question_id_str] = answer
                # Option 2: Skip this entry
                # print(f"Skipping entry with non-integer identifier: {question_id_str}")
                # continue
        else:
             print(f"Warning: Could not parse entry starting with: '{entry[:50]}...'")


    # Add a check for duplicate keys (if the VL model accidentally repeats a QUESTION tag)
    # This implementation implicitly takes the *last* occurrence found due to dict assignment.

    if not qa_dict:
         print("Warning: Resulting dictionary is empty after parsing.")

    return qa_dict

# New - Orchestration function for processing Answer Sheet (PDF or Image(s))
def process_answer_sheet_file(answer_sheet_path, question_paper_text_context, image_output_dir):
    """
    Handles PDF/Image input for answer sheet, performs OCR/Structuring via VL,
    and returns the structured dictionary.
    """
    image_paths = []
    if isinstance(answer_sheet_path, str) and answer_sheet_path.lower().endswith(".pdf"):
        image_paths = pdf_to_images(answer_sheet_path, output_folder=image_output_dir)
    elif isinstance(answer_sheet_path, str) and os.path.isfile(answer_sheet_path): # Single image file
        image_paths = [answer_sheet_path]
    elif isinstance(answer_sheet_path, list): # List of image files
        image_paths = answer_sheet_path
    else:
        print(f"ERROR: Invalid answer_sheet_path format: {answer_sheet_path}. Provide PDF path, image path, or list of image paths.")
        return None

    if not image_paths:
        print("ERROR: No images found or generated for the answer sheet.")
        return None

    # Extract text using VL model
    extracted_answer_text = extract_text_from_images(image_paths, question_paper_text_context)

    if not extracted_answer_text:
        print("ERROR: Failed to extract text from answer sheet images.")
        return None

    # Convert structured text to dictionary (ensure integer keys)
    student_answers_dict = convert_to_dict(extracted_answer_text)

    return student_answers_dict


# New - Orchestration function for processing Answer Key File (PDF or Image(s))
def process_key_file_to_dict(answer_key_path, question_paper_text_context, image_output_dir):
    """
    Handles PDF/Image input for answer key, performs OCR/Structuring via VL,
    and returns the structured dictionary {int: answer_string}.
    (Very similar to process_answer_sheet_file)
    """
    image_paths = []
    if isinstance(answer_key_path, str) and answer_key_path.lower().endswith(".pdf"):
        image_paths = pdf_to_images(answer_key_path, output_folder=image_output_dir)
    elif isinstance(answer_key_path, str) and os.path.isfile(answer_key_path): # Single image file
        image_paths = [answer_key_path]
    elif isinstance(answer_key_path, list): # List of image files
        image_paths = answer_key_path
    else:
        print(f"ERROR: Invalid answer_key_path format: {answer_key_path}. Provide PDF path, image path, or list of image paths.")
        return None

    if not image_paths:
        print("ERROR: No images found or generated for the answer key.")
        return None

    # Extract text using VL model - Use the same function, prompt should be general enough
    # Might consider a slightly different prompt if keys need different handling, but unlikely.
    print("\n--- Processing Answer Key File ---")
    extracted_key_text = extract_text_from_images(image_paths, question_paper_text_context)

    if not extracted_key_text:
        print("ERROR: Failed to extract text from answer key file.")
        return None

    # Convert structured text to dictionary (ensure integer keys)
    reference_answers_dict = convert_to_dict(extracted_key_text)
    print("Finished processing answer key file.")

    return reference_answers_dict

In [ ]:
# Code Cell 5: Grading Logic and Evaluation Functions
# ----------------------------------------------------
# All functions from the relevant part of Marks_Accuracy_model1.ipynb

# --- Answer Type Detection ---
def detect_answer_type(reference_text):
    """Detect the type of answer (code, mathematical, table, text) to adjust scoring weights."""
    if not isinstance(reference_text, str): return "text" # Handle non-string input

    # Compile regex patterns once (moved outside for efficiency, though function scope is ok)
    code_pattern = re.compile(r'\b(void|int|char|float|double|struct|class|def|import|while|for|if|else|return)\b', re.IGNORECASE)
    # More robust math pattern - includes common functions, greek letters (approximation), operators
    math_pattern = re.compile(r'(\$\$.*?\$\$|\\[a-zA-Z]+|\b(sin|cos|tan|log|exp|sqrt)\b|[+\-*/=<>≤≥^]|[ΣΠ∫]|[α-ωΑ-Ω])')
    table_pattern = re.compile(r'[|\+][-+]{3,}[|\+]') # Requires at least 3 dashes for a line

    # Check for code keywords (increase threshold slightly?)
    if len(code_pattern.findall(reference_text)) > 4: # Original was > 3
        return "code"

    # Check for mathematical content (increase threshold slightly?)
    if len(math_pattern.findall(reference_text)) > 6: # Original was > 5
        return "mathematical"

    # Check for tables (more robust check)
    if table_pattern.search(reference_text) or reference_text.count('\n|') > 1: # Look for multiple lines starting with |
        return "table"

    # Default
    return "text"

# --- Grading Weights ---
# Define weights as constants (from original code)
WEIGHTS = {
    "code": {
        'semantic_factual_similarity': 0.40,
        'code_structure': 0.30, # Specific metric for code
        'key_phrases': 0.10,
        'length_appropriateness': 0.10,
        'coherence': 0.05,
        'sequence_alignment': 0.05
    },
    "mathematical": {
        'semantic_factual_similarity': 0.50,
        'mathematical_expressions': 0.30, # Specific metric for math
        'key_phrases': 0.10,
        'length_appropriateness': 0.05,
        'coherence': 0.05
        # Sequence alignment might be less relevant for math? Original didn't include it.
    },
    "table": {
        'semantic_factual_similarity': 0.40,
        'table_structure': 0.30, # Specific metric for tables
        'key_phrases': 0.15,
        'length_appropriateness': 0.10,
        'coherence': 0.05
    },
    "text": {
        'semantic_factual_similarity': 0.65, # Higher weight for general text
        'length_appropriateness': 0.15,
        'key_phrases': 0.10,
        'coherence': 0.05,
        'sequence_alignment': 0.05
    },
    # New: Handle error case gracefully
    "error": {
        'semantic_factual_similarity': 0.0,
        'length_appropriateness': 0.0,
        'key_phrases': 0.0,
        'coherence': 0.0,
        'sequence_alignment': 0.0
    }
}

def get_weights_by_answer_type(answer_type):
    """Return appropriate weights based on answer type."""
    return WEIGHTS.get(answer_type, WEIGHTS["text"]) # Default to text weights if type unknown


# --- Core Metric Calculation Functions ---

# Function to wrap bert_score calculation for easier use and error handling
# New - Standardized BERTScore function
def calculate_bert_score(texts1, texts2, lang='en'):
    """Calculates BERTScore (P, R, F1) safely."""
    try:
        # Ensure inputs are lists of strings
        if isinstance(texts1, str): texts1 = [texts1]
        if isinstance(texts2, str): texts2 = [texts2]
        if not texts1 or not texts2 or not texts1[0] or not texts2[0]:
             # Handle empty input cases
             return 0.0, 0.0, 0.0

        # device = 'cuda' if torch.cuda.is_available() else 'cpu' # Optional: specify device
        P, R, F1 = bert_score.score(texts1, texts2, lang=lang, verbose=False, model_type='bert-base-uncased') # Specify model for consistency?
        # Return mean scores if multiple pairs were processed, or single score otherwise
        return P.mean().item(), R.mean().item(), F1.mean().item()
    except Exception as e:
        print(f"Error calculating BERTScore: {e}")
        # print(f"Texts involved: {texts1}, {texts2}") # Debugging
        return 0.0, 0.0, 0.0 # Return zero scores on error

def get_semantic_factual_similarity(student_text, reference_text):
    """Calculate combined semantic (BERTScore F1) and factual (number match) similarity."""
    if not isinstance(student_text, str) or not isinstance(reference_text, str): return 0.0

    # Semantic Similarity using BERTScore F1
    _, _, semantic_similarity_f1 = calculate_bert_score(student_text, reference_text)

    # Factual Accuracy based on numbers
    number_pattern = re.compile(r'\b\d+(?:\.\d+)?\b') # Match standalone numbers
    student_numbers = set(number_pattern.findall(student_text))
    reference_numbers = set(number_pattern.findall(reference_text))

    # Calculate intersection over union for numbers? Or just intersection / reference?
    # Original used intersection / reference. Stick to that unless specified otherwise.
    if not reference_numbers:
        factual_accuracy = 1.0 if not student_numbers else 0.5 # Both empty = 1.0; Student has extra = lower score?
    else:
        intersection_count = len(student_numbers.intersection(reference_numbers))
        factual_accuracy = intersection_count / len(reference_numbers)
        # Penalize slightly if student introduces numbers not in reference? Optional.
        # extra_numbers = len(student_numbers - reference_numbers)
        # penalty = extra_numbers * 0.1
        # factual_accuracy = max(0, factual_accuracy - penalty)

    # Combine scores (original weights: 70% semantic, 30% factual)
    combined_score = (semantic_similarity_f1 * 0.7 + factual_accuracy * 0.3)
    return max(0.0, min(1.0, combined_score)) # Ensure score is within [0, 1]

def check_length_ratio(student_text, reference_text):
    """Check if student answer length is appropriate relative to reference length (word count)."""
    if not isinstance(student_text, str) or not isinstance(reference_text, str): return 0.0

    try:
        student_length = len(word_tokenize(student_text))
        reference_length = len(word_tokenize(reference_text))
    except Exception as e:
        print(f"Error tokenizing for length ratio: {e}")
        return 0.0 # Return 0 if tokenization fails

    if reference_length == 0:
        return 0.0 if student_length > 0 else 1.0 # Perfect match if both empty, else 0

    ratio = student_length / reference_length

    # Original logic: score decreases if too long (beyond 1.5x) or too short.
    if ratio <= 1.5 and ratio >= 0.5: # Allow some flexibility (e.g., within 50%-150%)
         # Score proportional to ratio, capped at 1.0
         # Map ratio [0.5, 1.5] to score [0.5, 1.0] (roughly) - simpler approach below
         return min(1.0, ratio) if ratio <= 1.0 else (1.0 - (ratio - 1.0)*0.5) # Penalize length > 1.0
    elif ratio < 0.5:
        return ratio * 2 # Linear penalty for too short (0.5 ratio -> 1.0 score seems wrong, maybe just ratio?) Let's use ratio directly. --> return ratio
        # Corrected approach: return ratio if too short.
        return max(0.0, ratio) # Score is the ratio itself if too short
    else: # ratio > 1.5
        # Penalize excess length more sharply
        return max(0.0, 1.0 - (ratio - 1.5) * 0.5) # Example penalty: score drops from 1 at 1.5x

    # Simpler original logic interpretation:
    # if ratio <= 1.5: return min(1.0, ratio) # Score = ratio, capped at 1 if student is shorter
    # else: return min(1.0, 1.5 / ratio) # Score decreases inverse proportionally if student is longer

    # Let's stick to the *second interpretation* of the original logic as it's simpler:
    if reference_length == 0: return 1.0 if student_length == 0 else 0.0
    ratio = student_length / reference_length
    if ratio <= 1.0: # Student answer is shorter or equal
        # Score is proportional to length, up to reference length
        return ratio
    elif ratio <= 1.5: # Student answer is slightly longer (up to 1.5x) - Full score? Or slight penalty?
        # Original: min(1.0, 1.5/ratio) -> gives 1.0 at ratio=1.5. Let's use this.
         return 1.0 # Allow up to 1.5x length without penalty? Let's try this.
         # Original logic: return min(1.0, 1.5 / ratio) # Gives 1.0 at ratio 1.5, <1 above that.
    else: # ratio > 1.5
        # Score decreases
        return max(0.0, min(1.0, 1.5 / ratio)) # Original logic was likely this


def check_sequence_alignment(student_text, reference_text):
    """Check structural alignment (sentence and paragraph counts)."""
    if not isinstance(student_text, str) or not isinstance(reference_text, str): return 0.0

    try:
        student_sentences = sent_tokenize(student_text)
        reference_sentences = sent_tokenize(reference_text)
        # Handle potential tokenization issues leading to empty lists
        if not student_sentences and not reference_sentences: return 1.0
        if not student_sentences or not reference_sentences: return 0.0

        # Avoid division by zero - use max length as denominator
        max_sentences = max(len(student_sentences), len(reference_sentences))
        sent_ratio = min(len(student_sentences), len(reference_sentences)) / max_sentences if max_sentences > 0 else 1.0

        student_paragraphs = [p for p in student_text.split('\n\n') if p.strip()]
        reference_paragraphs = [p for p in reference_text.split('\n\n') if p.strip()]
        if not student_paragraphs and not reference_paragraphs: return 1.0
        if not student_paragraphs or not reference_paragraphs: return 0.0

        max_paragraphs = max(len(student_paragraphs), len(reference_paragraphs))
        para_ratio = min(len(student_paragraphs), len(reference_paragraphs)) / max_paragraphs if max_paragraphs > 0 else 1.0

        # Combine sentence and paragraph alignment scores (average)
        return (sent_ratio + para_ratio) / 2
    except Exception as e:
        print(f"Error during sequence alignment: {e}")
        return 0.0

def get_phrases(text, max_phrase_length=3):
    """Extract n-grams (phrases) up to max_phrase_length efficiently."""
    if not isinstance(text, str): return set()
    try:
        words = word_tokenize(text.lower())
    except Exception as e:
        print(f"Error tokenizing for phrases: {e}")
        return set()

    phrases = set()
    # Generate n-grams from 1 up to max_phrase_length
    for n in range(1, max_phrase_length + 1):
        for i in range(len(words) - n + 1):
            phrases.add(" ".join(words[i:i+n]))
    return phrases

def check_key_phrases(student_text, reference_text):
    """Check for presence of key phrases (including synonyms for single words within phrases)."""
    if not isinstance(student_text, str) or not isinstance(reference_text, str): return 0.0

    # Extract phrases (using n-grams up to length 3, as per original get_phrases)
    ref_phrases = get_phrases(reference_text, max_phrase_length=3)
    student_phrases = get_phrases(student_text, max_phrase_length=3)

    if not ref_phrases:
        return 1.0 if not student_phrases else 0.5 # No key phrases in reference

    # Expand reference phrases with synonyms (only for single-word phrases for efficiency, or based on original logic?)
    # Original code expanded synonyms *within* phrases up to 3 words. Let's replicate that carefully.
    expanded_ref_phrases = set(ref_phrases) # Start with original phrases
    phrases_to_expand = [p for p in ref_phrases if len(p.split()) <= 3] # Limit synonym expansion

    # This synonym expansion can be computationally expensive. Limit it.
    MAX_EXPANSIONS_PER_PHRASE = 5
    for phrase in phrases_to_expand:
        words = phrase.split()
        expansion_count = 0
        for i, word in enumerate(words):
            if expansion_count >= MAX_EXPANSIONS_PER_PHRASE: break
            synonyms = get_synonyms(word) # Uses cached function
            for synonym in synonyms:
                if expansion_count >= MAX_EXPANSIONS_PER_PHRASE: break
                if synonym != word: # Avoid adding the original word again
                    new_words = words[:i] + [synonym] + words[i+1:]
                    expanded_ref_phrases.add(" ".join(new_words))
                    expansion_count += 1


    # Direct phrase match score (intersection over reference set)
    intersection = student_phrases.intersection(expanded_ref_phrases)
    phrase_match_score = len(intersection) / len(expanded_ref_phrases) if expanded_ref_phrases else 1.0

    # Original code also included a BERTScore component for semantic phrase matching.
    # It calculated BERTScore between student *text* and a sample of *reference phrases*.
    # Let's replicate that part.

    # Select a limited number of reference phrases for BERTScore comparison to manage computation
    key_ref_phrases_for_bert = list(expanded_ref_phrases)[:20] # Limit to 20 representative phrases

    if not key_ref_phrases_for_bert:
         bert_match_score = 0.0 # Or 1.0 if student text is also empty? Assume 0.0
    else:
        # Calculate BERTScore F1 between the whole student answer and each key phrase
        _, _, bert_f1_scores = calculate_bert_score([student_text] * len(key_ref_phrases_for_bert), key_ref_phrases_for_bert)
        # Average the F1 scores? Or take max? Original used mean.
        bert_match_score = bert_f1_scores # calculate_bert_score already returns mean item

    # Combine direct match and semantic match (original weights: 40% direct, 60% semantic)
    combined_score = (phrase_match_score * 0.4 + bert_match_score * 0.6)
    return max(0.0, min(1.0, combined_score))


def check_coherence(text):
    """Check coherence using BERTScore between adjacent sentences."""
    if not isinstance(text, str): return 0.0
    try:
        sentences = sent_tokenize(text)
    except Exception as e:
        print(f"Error tokenizing for coherence: {e}")
        return 0.0

    if len(sentences) < 2:
        return 1.0 # Single sentence or empty text is considered coherent

    coherence_scores = []
    # Batching might be complex here due to pairs. Process sequentially.
    for i in range(len(sentences) - 1):
        # Calculate F1 score between adjacent sentences
        _, _, f1 = calculate_bert_score(sentences[i], sentences[i+1])
        coherence_scores.append(f1)

    # Average the scores
    return sum(coherence_scores) / len(coherence_scores) if coherence_scores else 1.0


# --- Type-Specific Metric Functions ---

# Compile patterns outside functions for efficiency
CODE_FUNCTION_PATTERN = re.compile(r'\b(void|int|char|float|double|struct|def|class)\s+(\w+)\s*\(.*\)\s*{?', re.IGNORECASE)
LOOP_PATTERN = re.compile(r'\b(while|for)\s*\(.*\)\s*{?', re.IGNORECASE)
CONDITIONAL_PATTERN = re.compile(r'\b(if|else\s+if|elif)\s*\(.*\)\s*{?', re.IGNORECASE)
VAR_DECL_PATTERN = re.compile(r'\b(int|char|float|double|struct|String|var|let|const)\s+\w+\s*(?:=.*;|;)', re.IGNORECASE) # Broader language support
MEMORY_ALLOC_PATTERN = re.compile(r'\b(malloc|calloc|realloc|new)\b', re.IGNORECASE)
COMPLEXITY_PATTERN = re.compile(r'\b(O|Θ|Ω)\s*\([^)]*\)', re.IGNORECASE) # Simpler complexity regex
FORMULA_PATTERN = re.compile(r'\b[a-zA-Z_][a-zA-Z0-9_]*\s*=[^;=\n]+(?:[+\-*/%^<>≤≥]|&&|\|\|)[^;=\n]+') # Basic formula/assignment check
TABLE_PATTERN = re.compile(r'[|\+][-+]{3,}[|\+]') # Requires 3+ dashes
TABLE_ROW_PATTERN = re.compile(r'^\s*\|.*\|\s*$', re.MULTILINE) # Match lines starting and ending with |

def analyze_code_structure(student_text, reference_text):
    """Analyze structural elements (functions, loops, conditionals, vars) in code."""
    if not isinstance(student_text, str) or not isinstance(reference_text, str): return 0.0

    def extract_code_elements(text):
        elements = {}
        # Function names (simple extraction)
        elements['functions'] = set(m.group(2) for m in CODE_FUNCTION_PATTERN.finditer(text))
        elements['loops'] = len(LOOP_PATTERN.findall(text))
        elements['conditionals'] = len(CONDITIONAL_PATTERN.findall(text))
        elements['variables'] = len(VAR_DECL_PATTERN.findall(text))
        elements['memory_ops'] = len(MEMORY_ALLOC_PATTERN.findall(text))
        return elements

    ref_elements = extract_code_elements(reference_text)
    student_elements = extract_code_elements(student_text)

    scores = {}
    # Function matching (Jaccard index for function names)
    ref_funcs = ref_elements['functions']
    student_funcs = student_elements['functions']
    if not ref_funcs and not student_funcs: scores['functions'] = 1.0
    elif not ref_funcs or not student_funcs: scores['functions'] = 0.0
    else:
        intersection = len(ref_funcs.intersection(student_funcs))
        union = len(ref_funcs.union(student_funcs))
        scores['functions'] = intersection / union if union > 0 else 1.0

    # Compare counts for other elements (loops, conditionals, vars, memory) using ratio logic
    for elem_type in ['loops', 'conditionals', 'variables', 'memory_ops']:
        ref_count = ref_elements.get(elem_type, 0)
        student_count = student_elements.get(elem_type, 0)

        if ref_count == 0:
            scores[elem_type] = 1.0 if student_count == 0 else 0.5 # Penalize extra elements
        else:
            ratio = student_count / ref_count
            # Use similar logic to length ratio: score = ratio if < 1, 1 if <= 1.5, decreases > 1.5
            if ratio <= 1.0: score = ratio
            elif ratio <= 1.5: score = 1.0
            else: score = max(0.0, 1.5 / ratio)
            scores[elem_type] = score

    # Average the scores for different elements
    final_score = sum(scores.values()) / len(scores) if scores else 0.0
    return max(0.0, min(1.0, final_score))


def compare_mathematical_expressions(student_text, reference_text):
    """Extract and compare mathematical formulas and complexity notations."""
    if not isinstance(student_text, str) or not isinstance(reference_text, str): return 0.0

    def extract_math_elements(text):
        elements = set()
        # Complexity notations
        elements.update(m.group(0).replace(" ", "").lower() for m in COMPLEXITY_PATTERN.finditer(text))
        # Basic formulas (normalize whitespace)
        elements.update(re.sub(r'\s+', '', m.group(0)).lower() for m in FORMULA_PATTERN.finditer(text))
        # Add other specific patterns if needed (e.g., summations, integrals - complex)
        return elements

    ref_expressions = extract_math_elements(reference_text)
    student_expressions = extract_math_elements(student_text)

    if not ref_expressions:
        return 1.0 if not student_expressions else 0.5 # No math expected

    # Compare using Jaccard index on the extracted sets
    intersection = len(student_expressions.intersection(ref_expressions))
    union = len(student_expressions.union(ref_expressions))

    return intersection / union if union > 0 else 1.0


def analyze_table_structure(student_text, reference_text):
    """Analyze table structure similarity (presence, rows, cols, headers)."""
    if not isinstance(student_text, str) or not isinstance(reference_text, str): return 0.0

    # Basic check for table presence
    student_has_table = bool(TABLE_PATTERN.search(student_text) or TABLE_ROW_PATTERN.search(student_text))
    ref_has_table = bool(TABLE_PATTERN.search(reference_text) or TABLE_ROW_PATTERN.search(reference_text))

    if not ref_has_table:
        return 1.0 if not student_has_table else 0.5 # Reference has no table

    if not student_has_table:
        return 0.0 # Reference expects table, student didn't provide

    # Compare Rows (count lines matching row pattern)
    student_rows = len(TABLE_ROW_PATTERN.findall(student_text))
    ref_rows = len(TABLE_ROW_PATTERN.findall(reference_text))
    if ref_rows == 0: row_score = 1.0 if student_rows == 0 else 0.5
    else: row_score = min(student_rows, ref_rows) / max(student_rows, ref_rows) if max(student_rows, ref_rows) > 0 else 1.0

    # Compare Columns (count separators in the first row found) - simplified
    student_first_row_match = TABLE_ROW_PATTERN.search(student_text)
    ref_first_row_match = TABLE_ROW_PATTERN.search(reference_text)
    student_cols = student_first_row_match.group(0).count('|') - 1 if student_first_row_match else 0
    ref_cols = ref_first_row_match.group(0).count('|') - 1 if ref_first_row_match else 0
    if ref_cols <= 0: col_score = 1.0 if student_cols <= 0 else 0.5
    else: col_score = min(student_cols, ref_cols) / max(student_cols, ref_cols) if max(student_cols, ref_cols) > 0 else 1.0

    # Compare Headers (optional, simple check if first row cells match somewhat)
    # This part is complex and was basic in the original. Keep it simple or omit?
    # Let's use a simple Jaccard on the first row cell contents.
    header_score = 0.5 # Default if cannot compare
    if student_first_row_match and ref_first_row_match:
        student_cells = set(c.strip().lower() for c in student_first_row_match.group(0).split('|')[1:-1]) # Cells between pipes
        ref_cells = set(c.strip().lower() for c in ref_first_row_match.group(0).split('|')[1:-1])
        if not ref_cells and not student_cells: header_score = 1.0
        elif not ref_cells or not student_cells: header_score = 0.0
        else:
             intersection = len(student_cells.intersection(ref_cells))
             union = len(student_cells.union(ref_cells))
             header_score = intersection / union if union > 0 else 1.0

    # Combine scores (e.g., 40% rows, 30% cols, 30% headers)
    final_score = (row_score * 0.4 + col_score * 0.3 + header_score * 0.3)
    return max(0.0, min(1.0, final_score))


# --- Single Answer Evaluation ---
def evaluate_single_answer(student_text, reference_text, total_marks):
    """Evaluate a student's answer against a reference using multiple weighted metrics."""
    details = {} # To store individual metric scores

    # Handle cases where either text is missing or not a string
    if not isinstance(student_text, str) or not student_text.strip():
        print("  - Student answer is empty or invalid.")
        # Assign score based on reference? If reference is also empty, maybe full marks?
        if not isinstance(reference_text, str) or not reference_text.strip():
             answer_type = "text" # or N/A?
             final_score = total_marks # Both empty, assume correct if marks > 0
             details = {'error': 'Both answers empty'}
        else:
             answer_type = detect_answer_type(reference_text)
             final_score = 0.0 # Student empty, reference not empty
             details = {'error': 'Student answer empty'}
        return {'final_score': round(final_score, 2), 'answer_type': answer_type, 'details': details}

    if not isinstance(reference_text, str) or not reference_text.strip():
        print("  - Reference answer is empty or invalid.")
        # Student provided something, reference empty. Partial marks? Or zero?
        answer_type = "text" # Detect type based on student answer?
        final_score = 0.0 # Cannot grade against empty reference
        details = {'error': 'Reference answer empty'}
        return {'final_score': round(final_score, 2), 'answer_type': answer_type, 'details': details}


    try:
        # Detect answer type based on REFERENCE answer
        answer_type = detect_answer_type(reference_text)
        print(f"  - Detected Reference Type: {answer_type}")

        # Calculate core metrics applicable to all types
        details['semantic_factual_similarity'] = get_semantic_factual_similarity(student_text, reference_text)
        details['length_appropriateness'] = check_length_ratio(student_text, reference_text)
        details['sequence_alignment'] = check_sequence_alignment(student_text, reference_text)
        details['key_phrases'] = check_key_phrases(student_text, reference_text)
        details['coherence'] = check_coherence(student_text) # Coherence of student answer

        # Calculate type-specific metrics
        if answer_type == "code":
            details['code_structure'] = analyze_code_structure(student_text, reference_text)
        elif answer_type == "mathematical":
            details['mathematical_expressions'] = compare_mathematical_expressions(student_text, reference_text)
        elif answer_type == "table":
            details['table_structure'] = analyze_table_structure(student_text, reference_text)

        # Get weights for the detected type
        weights = get_weights_by_answer_type(answer_type)

        # Calculate weighted score
        weighted_score = 0.0
        total_applied_weight = 0.0
        print("  - Calculating weighted score:")
        for metric, score in details.items():
            weight = weights.get(metric)
            if weight is not None: # Check if this metric has a weight defined for this type
                print(f"    - {metric}: Score={score:.3f}, Weight={weight:.2f}, Contribution={score * weight:.3f}")
                weighted_score += score * weight
                total_applied_weight += weight
            else:
                print(f"    - {metric}: Score={score:.3f} (No weight defined for type '{answer_type}')")


        # Normalize score by the sum of weights actually applied
        if total_applied_weight > 0:
            normalized_score = weighted_score / total_applied_weight
        else:
            normalized_score = 0.0 # Avoid division by zero if no weights applied
            print("  - Warning: Total applied weight is zero. Score set to 0.")

        # Ensure score is between 0 and 1
        final_normalized_score = max(0.0, min(1.0, normalized_score))

        # Scale score to total marks for the question
        scaled_score = final_normalized_score * total_marks
        # Ensure final score does not exceed total marks
        final_score = min(scaled_score, float(total_marks))

        print(f"  - Final Normalized Score (0-1): {final_normalized_score:.3f}")
        print(f"  - Final Score ({total_marks} marks): {final_score:.2f}")

        return {
            'final_score': round(final_score, 2),
            'answer_type': answer_type,
            'details': {k: round(v, 3) for k, v in details.items()} # Store rounded details
        }

    except Exception as e:
        print(f"ERROR evaluating answer: {e}")
        print(traceback.format_exc())
        return {
            'final_score': 0.0,
            'answer_type': 'error',
            'details': {'error': str(e)}
        }


# --- Overall Evaluation Orchestration ---
def evaluate_student_answers(student_answers: dict, reference_answers: dict, rubrics_marks: dict):
    """
    Evaluate multiple student answers against reference answers using detailed metrics.
    Ensures keys are integers for matching.
    """
    results = {}
    total_score_obtained = 0.0
    total_marks_possible = 0.0

    print("\n--- Starting Detailed Evaluation ---")

    # Standardize keys to integers for reliable matching
    try:
        student_answers_int = {int(k): v for k, v in student_answers.items()}
        reference_answers_int = {int(k): v for k, v in reference_answers.items()}
        rubrics_marks_int = {int(k): v for k, v in rubrics_marks.items()}
    except ValueError as e:
        print(f"ERROR: Could not convert all keys in input dictionaries to integers. Please ensure keys are numeric. Details: {e}")
        return None # Cannot proceed if keys are not consistently numeric

    # Use keys from the rubrics/marks dictionary as the canonical list of questions to grade
    qids_to_grade = sorted(rubrics_marks_int.keys())
    print(f"Questions to be evaluated (based on marks provided): {qids_to_grade}")

    for qid in qids_to_grade:
        print(f"\n--- Evaluating Question {qid} ---")

        student_answer = student_answers_int.get(qid)
        reference_answer = reference_answers_int.get(qid)
        question_marks = rubrics_marks_int.get(qid) # Already checked it exists

        # Handle missing student or reference answer for a question defined in marks
        if student_answer is None:
            print(f"  - Warning: No student answer found for question {qid}. Awarding 0 marks.")
            evaluation = {'final_score': 0.0, 'answer_type': 'missing', 'details': {'error': 'Student answer missing'}}
        elif reference_answer is None:
            print(f"  - Warning: No reference answer found for question {qid}. Cannot grade. Awarding 0 marks.")
            evaluation = {'final_score': 0.0, 'answer_type': 'missing', 'details': {'error': 'Reference answer missing'}}
        else:
            # Evaluate the answer
            evaluation = evaluate_single_answer(
                student_answer,
                reference_answer,
                question_marks,
                # Note: BERTScore calculation is now inside get_semantic_factual_similarity etc.
            )

        results[qid] = evaluation
        total_score_obtained += evaluation.get('final_score', 0.0) # Add score, default to 0 if 'final_score' missing
        total_marks_possible += question_marks

    # Calculate overall percentage safely
    percentage = round((total_score_obtained / total_marks_possible) * 100, 2) if total_marks_possible > 0 else 0.0

    print("\n--- Detailed Evaluation Complete ---")

    return {
        'question_results': results,
        'overall_score': round(total_score_obtained, 2),
        'total_marks': total_marks_possible,
        'percentage': percentage
    }

In [ ]:
# Code Cell 6: Main Execution Workflow
# -----------------------------------

print("==============================================")
print("  Automated Assignment Grading System (v2)  ")
print("==============================================")

# --- Preparations ---
# Ensure API client is ready
if not client:
    print("ERROR: OpenRouter API client failed to initialize. Grading cannot proceed.")
    # Exit or raise error
    # exit() # Uncomment to stop execution

# Create temporary directory for images if needed
os.makedirs(IMAGE_OUTPUT_FOLDER, exist_ok=True)

# --- Step 1: Process Question Paper ---
print("\n[Step 1/5] Processing Question Paper...")
question_paper_text = extract_text_from_question_paper(QUESTION_PAPER_PATH)

if not question_paper_text:
    print("ERROR: Failed to process question paper. Cannot proceed.")
    # exit() # Uncomment to stop execution
else:
    # Display snippet of extracted QP text (optional)
    print("Question Paper Text Snippet:")
    print("-" * 30)
    print(question_paper_text[:500] + "...")
    print("-" * 30)

# --- Step 2: Process Student Answer Sheet ---
print("\n[Step 2/5] Processing Student Answer Sheet...")
student_answers_dict = process_answer_sheet_file(
    ANSWER_SHEET_PATH,
    question_paper_text, # Provide QP text as context for VL model
    IMAGE_OUTPUT_FOLDER
)

if not student_answers_dict:
    print("ERROR: Failed to process student answer sheet. Cannot proceed.")
    # exit() # Uncomment to stop execution
else:
    print(f"Successfully processed student answer sheet. Found answers for {len(student_answers_dict)} questions.")
    # Display snippet (optional)
    # print("Sample Student Answer (Q1):", student_answers_dict.get(1, "N/A")[:200] + "...")


# --- Step 3: Load or Process Answer Key ---
print("\n[Step 3/5] Loading/Processing Answer Key...")
final_reference_answers_dict = None

if ANSWER_KEY_SOURCE == 'file':
    final_reference_answers_dict = process_key_file_to_dict(
        ANSWER_KEY_PATH,
        question_paper_text, # Provide QP context here too if needed
        IMAGE_OUTPUT_FOLDER
    )
elif ANSWER_KEY_SOURCE == 'dict':
    # Ensure keys are integers if using predefined dict
    try:
        final_reference_answers_dict = {int(k): v for k, v in ANSWER_KEY_DICT.items()}
        print("Using predefined answer key dictionary.")
    except ValueError:
        print("ERROR: Keys in predefined ANSWER_KEY_DICT must be integers.")
        final_reference_answers_dict = None
else:
    print(f"ERROR: Invalid ANSWER_KEY_SOURCE '{ANSWER_KEY_SOURCE}'. Use 'file' or 'dict'.")

if not final_reference_answers_dict:
    print("ERROR: Failed to load or process answer key. Cannot proceed.")
    # exit() # Uncomment to stop execution
else:
     print(f"Answer key ready. Contains answers for {len(final_reference_answers_dict)} questions.")
     # Display snippet (optional)
     # print("Sample Reference Answer (Q1):", final_reference_answers_dict.get(1, "N/A")[:200] + "...")


# --- Step 4: Prepare Marks Dictionary ---
print("\n[Step 4/5] Preparing Marks Rubric...")
# Ensure keys are integers
try:
    final_marks_dict = {int(k): v for k, v in MARKS_PER_QUESTION.items()}
    print(f"Marks rubric loaded for {len(final_marks_dict)} questions.")
except ValueError:
    print("ERROR: Keys in MARKS_PER_QUESTION must be integers.")
    final_marks_dict = None

if not final_marks_dict:
     print("ERROR: Failed to load marks rubric. Cannot proceed.")
     # exit()

# --- Step 5: Perform Grading ---
print("\n[Step 5/5] Performing Evaluation...")
evaluation_results = None
if student_answers_dict is not None and final_reference_answers_dict is not None and final_marks_dict is not None:
    evaluation_results = evaluate_student_answers(
        student_answers=student_answers_dict,
        reference_answers=final_reference_answers_dict,
        rubrics_marks=final_marks_dict
    )
else:
    print("Skipping evaluation due to errors in previous steps.")


# --- Step 6: Display Results ---
print("\n==============================================")
print("             Grading Results")
print("==============================================")

if evaluation_results:
    print(f"\nOverall Score: {evaluation_results['overall_score']} / {evaluation_results['total_marks']}")
    print(f"Percentage: {evaluation_results['percentage']}%")
    print("\n--- Scores per Question ---")
    for qid, result in sorted(evaluation_results['question_results'].items()):
        marks = final_marks_dict.get(qid, "N/A")
        print(f"\nQuestion {qid} (Max Marks: {marks}, Type: {result.get('answer_type', 'N/A')})")
        print(f"  Score Awarded: {result.get('final_score', 'Error')}")
        print("  Details:")
        if 'details' in result and isinstance(result['details'], dict):
            for metric, score in result['details'].items():
                if metric != 'error': # Don't reprint error here
                     print(f"    - {metric:<30}: {score:.3f}")
                else:
                     print(f"    - Error during evaluation: {score}")
        elif 'final_score' in result and result['final_score'] == 0.0:
             print("    - Evaluation skipped or failed (Score 0).")
        else:
             print("    - No details available.")

else:
    print("Evaluation could not be completed due to errors.")

print("\n==============================================")
print("              End of Report")
print("==============================================")

# Optional: Clean up temporary image folder
# import shutil
# if os.path.exists(IMAGE_OUTPUT_FOLDER):
#     print(f"\nCleaning up temporary image folder: {IMAGE_OUTPUT_FOLDER}")
#     # shutil.rmtree(IMAGE_OUTPUT_FOLDER)

  Automated Assignment Grading System (v2)  

[Step 1/5] Processing Question Paper...
Extracting text from Question Paper: question_paper.jpeg using meta-llama/llama-4-maverick:free...
Question paper text extracted successfully.
Question Paper Text Snippet:
------------------------------
QUESTIONS:

QUESTION 1:
Explain asymptotic notations used to represent the time complexity. [03]

QUESTION 2:
Write a pseudo code to delete duplicate nodes from the unsorted Singly Linked List. Perform sorting on the resultant list and display the same. [07]

OR

Write an algorithm to perform the following operations on Circular Linked List and explain the same with an example.
1. Insertion at the start
2. Deletion of last node [07]

QUESTION 3:
Convert given INFIX expression to POSTFIX with the...
------------------------------

[Step 2/5] Processing Student Answer Sheet...
Converting PDF 'Joel-D041.pdf' to images in 'temp_images_output'...
Error converting PDF '/content/Joel-D041.pdf' to images: Unab